In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
from collections import defaultdict
import re

%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### good machine learning repository:
https://github.com/susanli2016/Machine-Learning-with-Python

### some basics
this is a good reference: 

https://www.hackerearth.com/practice/machine-learning/prerequisites-of-machine-learning/bayes-rules-conditional-probability-chain-rule/tutorial/

* ### 1. Product rule

P(X, Y) = P(X|Y)* P(Y)

P(X, Y) is the join probability both X and Y will occur. 

P(Y) is the probability that Y occurs.

P(X|Y) is conditional probability. the probability that X occurs given that Y has already occurred.

Product rule states that the probability that both X and Y will occur is the product of the probability that Y occurs and the probability that X occurs given that Y has already occurred.

* ### 2. chain rule

Generalizing the product rule leads to chain rule. 

P(A,B) = p(A|B) p(B)

We can extend this for three variables:

P(A,B,C) = P(A| B,C) P(B,C) = P(A|B,C) P(B|C) P(C)

and in general to n variables:

P(A1, A2, ..., An) = P(A1| A2, ..., An) P(A2| A3, ..., An) P(An-1|An) P(An)

* ### 3. Bayes' theorem

Bayes rule lets us to update our belief as a event occurs (e.g. new evidence discovered, new information about the event we are trying to predict). 

$ P(A|B) = \frac{P(B|A)P(A)}{P(B)}$

It is more intuitive to look at an example. 
event A: having cancer

event B: smoke

P(A): probability of a person having cancer. suppose we know cancer occurs in 5% general population. so P(A)=0.05. If we want to predict the chance of a person having cancer, we can just say the probability of this person having 5% wihtout knowing any additional information about this person.

P(B): probability of a person who smokes. Suppose we know a little more background about this person that he has been a smoker. We now should be able to update our belief and give more accurate prediction P(A|B) based on this new information based on Bayes rule.

P(A|B): probability of the person has cancer given that he is a smoker. This is a conditional probability, also called posterior.

P(B|A): probability of being a smoker given that the person has cancer.

We are trying to estimate the probability of a person having cancer P(A). Since 5% population has cancer, we can just assume the probability that a person having cancer is 0.05. Research has suggested that smoking links to cancer. And we also know the person is a smoker. So this new knowledge (evidence) should allow us to make a more accurate prediction according to Bayes rule. We now know:
P(A) = 0.05: 5% population have cancer.
P(B) = 0.1: 10% population are smokers.
P(B|A) = 0.2: 20% of people who have cancer are smokers.
$P(A|B) = \frac{P(B|A)*P(A)}{P(B)} = \frac{0.2 \times 0.05}{0.1} = 10\% $ 





In this problem, we look at maximum likelihood parameter estimation using the naive
Bayes assumption. Here, the input features $x_j , j = 1, . . . , n$ to our model are discrete,
binary-valued variables, so $x_j ∈ {0, 1}$. We call $x = [x_1 x_2 · · · x_n]^T$
to be the input vector.
For each training example, our output targets are a single binary-value y ∈ {0, 1}. 

Our
model is then parameterized by 
\begin{align*}
\begin{split}
ϕ_{j|y=0} &= p(x_j = 1|y = 0), \\
ϕ_{j|y=1} &= p(x_j = 1|y = 1), and\\
ϕ_y &= p(y = 1). 
\end{split}
\end{align*}

We model the joint distribution of (x, y) according to
\begin{align*}
\begin{split}
p(y) &= (ϕ_y)^y(1 − ϕ_y)^{1−y}\\
p(x|y = 0) &= \prod^n_{j=1}p(x_j|y = 0)\\
&=\prod^n_{j=1}(ϕ_{j|y=0})^{x_j}(1 − ϕ_{j|y=0})^{1−x_j}\\
p(x|y = 1) &= \prod^n_{j=1}p(x_j |y = 1)\\
&=\prod^n_{j=1}(ϕ_{j|y=1})^{x_j}(1 − ϕ_{j|y=1})^{1−x_j}
\end{split}
\end{align*}

(a) Find the joint likelihood function $ℓ(φ) = log \prod^m_{i=1} p(x^{(i)}, y^{(i)}; φ)$ in terms of the model parameters given above. Here, φ represents the entire set of parameters {$ϕ_y, ϕ_{j|y=0}, ϕ_{j|y=1}, j = 1, . . . , n$}.

### implementation of Naive Bayes algorithm for a text classification problem
here we use the 20 newsgroups dataset. The dataset has news documents partitioned to 20 different groups based on topics.

http://qwone.com/~jason/20Newsgroups/

code is adapted from https://github.com/aishajv/Unfolding-Naive-Bayes-from-Scratch for my learning purposes.

In [ ]:
from sklearn.datasets import fetch_20newsgroups


categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med'] 
train = fetch_20newsgroups(subset = 'train',categories = categories)

In [31]:
train_data = np.array(train.data) 
train_labels = np.array(train.target) 

In [52]:
train_data[1]

"From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the problem:\n\tI have a rectangular mesh in the uv domain, i.e  the mesh is a \n\tmapping of a 3d Bezier patch into 2d. The area in this domain\n\twhich is inside a trimming loop had to be rendered. The trimming\n\tloop is a set of 2d Bezier curve segments.\n\tFor the sake of notation: the mesh is made up of cells.\n\n\tMy problem is this :\n\tThe trimming area has to be split up into individual smaller\n\tcells bounded by the trimming curve segments. If a cell\n\tis wholly inside the area...then it is output as a whole ,\n\telse it is trivially rejected. \n\n\tDoes any body know how thiss can be done, or is there any algo. \n\tsomewhere for doing this.\n\n\tAny help would be appreciated.\n\n\tThanks, \n\tAni.\

In [54]:
len(train_data)
len(train_labels)
process_text(train_data[3])
train_labels[:1]
train.target_names

2257

2257

'from s let rug nl m m zwart subject catholic church poland organization faculteit der letteren rijksuniversiteit groningen nl lines hello i m writing a paper on the role of the catholic church in poland after can anyone tell me more about this or fill me in on recent books articles in english german or french most important for me is the role of the church concerning the abortion law religious education at schools birth control and the relation church state government thanx masja m m zwart s let rug nl '

array([1])

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [8]:
pd.options.display.max_colwidth=250
pd.DataFrame(data=np.column_stack([train_data,train_labels]),columns=["Training Examples","Training Labels"]).head()

,Training Examples,Training Labels
0,From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert ...,1
1,"From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can he...",1
2,"From: djohnson@cs.ucsd.edu (Darin Johnson)\nSubject: Re: harrassed at work, could use some prayers\nOrganization: =CSE Dept., U.C. San Diego\nLines: 63\n\n(Well, I'll email also, but this may apply to other people, so\nI'll post also.)\n\n>I've b...",3
3,"From: s0612596@let.rug.nl (M.M. Zwart)\nSubject: catholic church poland\nOrganization: Faculteit der Letteren, Rijksuniversiteit Groningen, NL\nLines: 10\n\nHello,\n\nI'm writing a paper on the role of the catholic church in Poland after 1989. \n...",3
4,"From: stanly@grok11.columbiasc.ncr.com (stanly)\nSubject: Re: Elder Brother\nOrganization: NCR Corp., Columbia SC\nLines: 15\n\nIn article <Apr.8.00.57.41.1993.28246@athos.rutgers.edu> REXLEX@fnal.gov writes:\n>In article <Apr.7.01.56.56.1993.228...",3


### build the Naive Bayes model

In [7]:
d = np.array([defaultdict(int) for index in range(2)])
d[0].items()
d[0][0]

dict_items([])

0

In [121]:
class NaiveBayes:
    
    def __init__(self, classes):        
        self.classes = classes  
        
    def process_text(self, txts):
        '''
        replace non-alphabets to spaces, replace multiple spaces to single space, convert to lower cases
        '''
        txts = re.sub('[^a-z\s]+',' ', txts, flags = re.IGNORECASE) 
        txts = re.sub('(\s+)',' ',txts) 
        txts = txts.lower()  
        return txts 

    def count_words(self, txts, clss):
        '''
        build bag of words dictionary, bow[clss][word]: word_counts, clss=class
        '''   
        txts = txts[0] # not sure why txts is a np array
        for word in txts.split():       
            bows[clss][word] += 1 


    def train(self, train_data, train_labels):
        '''
        build bag of words dictionary with nested keys [clss][word]
        '''           
        #constructing BoW for each class
        for ix, clss in enumerate(self.classes):          
            clss_data = train_data[train_labels==clss] 
            clss_data = pd.DataFrame([self.process_text(txts) for txts in clss_data])
            #now costruct BoW of this particular category
            np.apply_along_axis(self.count_words, 1, clss_data, ix)
        return bows       
    

    def class_priors(self, train_labels):
        class_priors = np.empty(self.classes.shape[0])
        for i, clss in enumerate(self.classes):           
            #Calculating prior probability p(c) for each class
            class_priors[i]=np.sum(train_labels == clss)/float(train_labels.shape[0]) 
        return class_priors  
    

    def class_wcounts(self, bows):
        class_wcounts = np.empty(self.classes.shape[0])
        for i, _ in enumerate(self.classes): 
            class_wcounts[i] = np.sum(np.array(list(bows[i].values())))+1 
        return class_wcounts
    

    def vocabs(self, bows):    
        vocabs = []   
        for i, _ in enumerate(self.classes):  
            vocabs += bows[i].keys()    
        vocabs = np.unique(np.array(vocabs))
        return vocabs

    def denominator(self, class_wcounts, vocabs):
        denoms = np.array([class_wcounts[i] + len(vocabs) + 1 for i, _ in enumerate(self.classes)])                                                                          
        return denoms
    

    def model_params(self, bows, class_priors, denoms):
        mps = [(bows[i], class_priors[i], denoms[i]) for i, _ in enumerate(self.classes)]                               
        mps = np.array(mps)  
        return mps

    
    def predict(self, txts, mps): 
        ''' 
        probability of test example in ALL CLASSES
        mps, model params tuble (bows, class_wcounts, class_denominators)
        for each word, compute likelihood [ count(w|c)+1 ] / [ count(c) + |V| + 1 ] 
        get count of this test word in training set for each class  
        log to prevent underflow! likelihood are all very small fractions, product to sum.
        likelihood estimate of the given test texts against every class 
        but we need posterior probility
        '''                                  
        likelihood = np.zeros(self.classes.shape[0])       
        for i, _ in enumerate(self.classes):
            for word in txts.split():     
                wcounts_inmodel = mps[i][0].get(word,0)+1                
                class_denom = float(mps[i][2])                              
                word_prob = wcounts_inmodel/class_denom                  
                likelihood[i] += np.log(word_prob)  
        post_prob = np.empty(self.classes.shape[0])
        for i, _ in enumerate(self.classes):
            post_prob[i] = likelihood[i] + np.log(mps[i][1])
        return post_prob
    

    def test(self, test_set, mps):
        '''
        Determines probability of each test txts against all classes and predicts the label
        against which the class probability is maximum
        Returns:
        preds of test txtss - A single prediction against every test txts
        '''  
        preds = [] 
        for txts in test_set:             
            txts = process_text(txts)
            #simply get the posterior probability of every txts                                  
            post_prob = self.predict(txts, mps) 
            preds.append(self.classes[np.argmax(post_prob)])
        return np.array(preds) 

In [122]:
import pprint
classes = np.unique(train_labels)
bows = np.array([defaultdict(lambda:0) for index in range(classes.shape[0])])
nb = NaiveBayes(np.unique(train_labels))
bows = nb.train(train_data, train_labels)
class_priors = nb.class_priors(train_labels)
class_priors
class_wcounts = nb.class_wcounts(bows)
class_wcounts
vocabs = nb.vocabs(bows)
vocabs[:10]
denoms = nb.denominator(class_wcounts, vocabs)
denoms
mps = nb.model_params(bows, class_priors, denoms)
mps[0][1]
mps[0][2]
nb.predict('saf dlfd 877&&* ff!', mps)

array([0.21267169, 0.25875055, 0.26318121, 0.26539654])

array([171596., 136413., 183395., 226217.])

array(['a', 'aa', 'aaa', 'aaaa', 'aaai', 'aacc', 'aad', 'aah', 'aalborg',
       'aamrl'], dtype='<U78')

array([202782., 167599., 214581., 257403.])

0.21267168808152415

202782.0

array([-50.42755283, -49.4692088 , -50.44068273, -51.1601231 ])

In [126]:
ngs_test = fetch_20newsgroups(subset='test',categories=categories) 
test_data = ngs_test.data 
test_labels = ngs_test.target
print ("Number of Test Examples: ",len(test_data))
print ("Number of Test Labels: ",len(test_labels))

    


Number of Test Examples:  1502
Number of Test Labels:  1502


In [127]:
class_preds = nb.test(test_data, mps) 

#check how many predcitions actually match original test labels
test_acc = np.sum(class_preds == test_labels)/float(test_labels.shape[0]) 

print ("Test Set Examples: ",test_labels.shape[0])
print ("Test Set Accuracy: ",test_acc*100,"%")

Test Set Examples:  1502
Test Set Accuracy:  93.87483355525966 %


Finally, we made it. the Naive Bays model is not that naive considering that it has over 93% test accuracy. Yeah!!!

to understand python object and self
https://micropyramid.com/blog/understand-self-and-__init__-method-in-python-class/

* class: a set of things (objects) having some properties in common and are different from others by type, kind, qulity etc. class is the blueprint for individual object with exactly the same behaviours.
* object: one of the instances (examples) of the class, which can perform the functionalities defined in the class.
* self: represent the instance of the class. by using self, we can access the attributes and methods of the class. 
* __init__: is the contructor, we call this reserved method to create an object from a class. It allows the class to initialize the attributes of a class.

In [9]:
class Rectangle:
   def __init__(self, length, breadth, unit_cost=0):
       self.length = length
       self.breadth = breadth
       self.unit_cost = unit_cost
   
   def get_perimeter(self):
       return 2 * (self.length + self.breadth)
   
   def get_area(self):
       return self.length * self.breadth
   
   def calculate_cost(self):
       area = self.get_area()
       return area * self.unit_cost
# breadth = 120 cm, length = 160 cm, 1 cm^2 = Rs 2000
r = Rectangle(160, 120, 2000)
print("Area of Rectangle: %s cm^2" % (r.get_area()))
print("Cost of rectangular field: Rs. %s " %(r.calculate_cost()))

Area of Rectangle: 19200 cm^2
Cost of rectangular field: Rs. 38400000 


In [10]:
r = Rectangle(160, 120, 2000)